In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [ ]:
# Cargar los datasets
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

print("Dimensiones del set de entrenamiento:", train_df.shape) # Imprimimos para ver
print("Dimensiones del set de prueba:", test_df.shape) # Imprimimos para ver

# Vistazo rápido a los primeros registros del set de entrenamiento
print("\nPrimeras filas del set de entrenamiento:")
train_df.head() # Imprimimos para ver

Dimensiones del set de entrenamiento: (891, 12)
Dimensiones del set de prueba: (418, 11)

Primeras filas del set de entrenamiento:


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


a) Describir brevemente la cantidad de valores faltantes para cada una de las variables de la base de datos.
---

*Para identificar cuántos datos nos faltan por columna, usaremos la función .isnull().sum(). Esto nos dará un conteo de los valores nulos (NaN) en cada variable tanto para el set de entrenamiento como para el de prueba.*

In [ ]:
# Calcular valores faltantes para el set de entrenamiento
missing_train = train_df.isnull().sum()
missing_train = missing_train[missing_train > 0] # Filtrar solo las columnas con 0 par a que sea mas limpio
print("Valores faltantes en train.csv:") # Imprimimos para ver
print(missing_train)
print("-" * 30) # Separamos con _

# Calcular valores faltantes para el set de prueba
missing_test = test_df.isnull().sum()
missing_test = missing_test[missing_test > 0] # Filtrar solo las columnas con 0
print("Valores faltantes en test.csv:") # Imprimimos para ver
print(missing_test)

Valores faltantes en train.csv:
Age         177
Cabin       687
Embarked      2
dtype: int64
------------------------------
Valores faltantes en test.csv:
Age       86
Fare       1
Cabin    327
dtype: int64


b) Completar aquellas variables que se encuentran faltantes para las bases de train (“Age”,”fare”, “Cabin”) y test ( “Age”, “Cabin”, “embarked”).

---


1.   *Ahora procedemos a imputar los datos faltantes. Usamos estrategias distintas según lo que vimos en clase.*
2.   *Age: Rellenamos los valores faltantes con la mediana de la edad. La mediana es más robusta a valores atípicos que la media.*
3.   *Fare: Solo falta 1 valor en el set no me parece importante. Usamos también la mediana.*
4.   *Embarked: Rellenaremos los dos valores faltantes en el set de entrenamiento con la moda (el puerto de embarque más común).*
5.   *Cabin: Esta variable tiene demasiados valores faltantes. Como estrategia común asigno faltantes con una categoría propia, como 'U' de 'Unknown', para no perder la información de quién tenía una cabina asignada y quién no, pero para este caso lo voy a desestimar.*


##NOTA: La consigna dice que el valor faltante de Embarked y Fare estan en los dataset contrarios respectivamente:

*   Valores faltantes en train.csv:
*   Age         177
*   Cabin       687
*   Embarked      2
*   ------------------------------
*   Valores faltantes en test.csv:
*   Age       86
*   Fare       1
*   Cabin    327







In [ ]:
# --- Imputación para el set de ENTRENAMIENTO (train.csv) ---

# Rellenar 'Age' con la mediana
median_age_train = train_df['Age'].median()
train_df['Age'].fillna(median_age_train, inplace=True)

# Rellenar 'Embarked' con la moda
mode_Embarked_train = train_df['Embarked'].mode()[0]
train_df['Embarked'].fillna(mode_Embarked_train, inplace=True)

# Rellenar 'Cabin' con 'U' solo para que de 0 porque despues lo borro
train_df['Cabin'].fillna('U', inplace=True)


# --- Imputación para el set de PRUEBA (test.csv) ---

# Rellenar 'Age' con la mediana
median_age_test = test_df['Age'].median()
test_df['Age'].fillna(median_age_test, inplace=True)

# Rellenar 'Fare' con la mediana
median_Fare_test = test_df['Fare'].median()
test_df['Fare'].fillna(median_Fare_test, inplace=True)

# Rellenar 'Cabin' con 'U' solo para que de 0 porque despues lo borro
test_df['Cabin'].fillna('U', inplace=True)

# --- Verificación --- Imprimimos para ver
# Debe quedar en 0

print("\nValores faltantes en train.csv después de la imputación (entrenamiento):")
print(train_df.isnull().sum())
print("\nValores faltantes en test.csv después de la imputación (prueba):")
print(test_df.isnull().sum())


Valores faltantes en train.csv después de la imputación (entrenamiento):
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

Valores faltantes en test.csv después de la imputación (prueba):
PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64


c) Ajustar el primer modelo de regresión logística

---
*Para ajustar el modelo, primero debemos preparar los datos. Debemos convertir las variables categóricas (Sex, Embarked) en variables numéricas. Usaremos la técnica de One-Hot Encoding.*
*También seleccionaremos un subconjunto de variables (features) para entrenar el modelo.*

In [1]:
# Convertir variables categóricas a numéricas
train_df['Sex'] = train_df['Sex'].map({'male': 0, 'female': 1})
test_df['Sex'] = test_df['Sex'].map({'male': 0, 'female': 1})

# Eliminar columnas que no usaremos en el modelo inicial
train_df_model = train_df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])
test_df_model = test_df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])

# Convertir variables categóricas a numéricas usando One-Hot Encoding
train_df_processed = pd.get_dummies(train_df_model, columns=['Embarked'], drop_first=True)
test_df_processed = pd.get_dummies(test_df_model, columns=['Embarked'], drop_first=True)

# Definir nuestras variables (X) y nuestro objetivo (y)
X = train_df_processed.drop('Survived', axis=1)
y = train_df_processed['Survived']

# El set de prueba final será test_df_processed
X_final_test = test_df_processed

print("Variables listas para el modelo:") # Imprimimos para ver
display(X.head())

NameError: name 'train_df' is not defined

d) Entrenar y determinar el nivel de accuracy del primer modelo

---
*Para evaluar la precisión (accuracy) de nuestro modelo de manera fiable, no podemos usar el set de prueba (test.csv) porque no tiene la columna Survived. Dividimos nuestro set de entrenamiento (train.csv) en dos: uno para entrenar el modelo y otro para validarlo.*

*Usamos train_test_split para separar un 80% de los datos para entrenamiento y un 20% para prueba y validación.*

In [ ]:
# 1. Dividir el set de entrenamiento en entrenamiento y validación (80/20)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Modelo de Regresión Logística
log_reg_model = LogisticRegression(max_iter=1000) # Ponemos 1000 max_iter para asegurar convergencia

# 3. Entrenamos el modelo con los datos de entrenamiento
log_reg_model.fit(X_train, y_train)

# 4. Predesimos sobre el conjunto de validación
y_pred_val = log_reg_model.predict(X_val)

# 5. Calcular accuracy del modelo
accuracy = accuracy_score(y_val, y_pred_val)

print(f"El nivel de accuracy del primer modelo de regresión logística es: {accuracy * 100:.2f}%") # Imprimimos para ver

El nivel de accuracy del primer modelo de regresión logística es: 81.01%


In [ ]:
#Validacion cruzada X5
scores = cross_val_score(log_reg_model, X_train, y_train, cv=5)
print(f"Precision de la validacion cruzada: {scores}")

Precision de la validacion cruzada: [0.81818182 0.81118881 0.78873239 0.73943662 0.79577465]%


In [ ]:
train_df_model['Survived'].value_counts(normalize=True) * 100


,proportion
Survived,
0,61.616162
1,38.383838
